In [1]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import yaml
import os
import sys

In [2]:
with open('D:\\Documents\\data_sources\\is2_creds.yml', 'r') as cred_file:
    cred = yaml.load(cred_file)


In [3]:
login_payload = {
    'id__email': cred['username'],
    'id__password': cred['password']
}

In [4]:
def get_token(session, login_url='https://clients.insightsquared.com/login/'):
    # get the unique token code
    login_resp = session.get(login_url)
    token = BeautifulSoup(login_resp.text, 'lxml').find('input', attrs={'name': 'csrfmiddlewaretoken'}).attrs['value'] 
    return token

def get_client(base_page):
    client_picker = BeautifulSoup(base_page.text, 'lxml').find('client-picker').attrs['params']
    client_yaml = yaml.load(client_picker[16:].replace('}', '') + '}')  # need to deal with crummy string formatting
    # client_yaml['abbreviation'], client_yaml['id'], client_yaml['name'], client_yaml['site'] 
    return client_yaml

def login(payload=login_payload):
    # begin a web session and visit the IS2 webpage
    session = requests.session()
    login_url = 'https://clients.insightsquared.com/login/'
    token = get_token(session)
    payload['csrfmiddlewaretoken'] = token  # append token to payload dict
    # get an authenticated session and verity results
    login_response = session.post(
        login_url, 
        data=payload, 
        headers=dict(referer=login_url)
    )
    # check that the login worked
    print BeautifulSoup(login_response.text, 'lxml').find('title')
    print get_client(login_response)
    return session, token

In [23]:
session, token = login()

<title> My Dashboard | InsightSquared</title>
{'abbreviation': 'is2dev', 'id': 289, 'name': 'InsightSquared Development', 'site_url': 'https://clients.insightsquared.com/'}


In [24]:
token

'RHkvDPEwzDdoopkamK4ldowN0rA8GGFL'

In [25]:
url = 'https://clients.insightsquared.com/settings/tags/Employee/'

In [26]:
filters_page = session.get(url)

In [27]:
soup = BeautifulSoup(filters_page.text, 'lxml')
soup.find('title')

<title> Tag Management | InsightSquared</title>

In [ ]:
# onclick="IS.Tag.newTag()
# onsubmit="IS.Tag.newTagSubmit()

In [28]:
payload = {
    #'onclick': 'IS.Tag.newTag()',
    'new_tag_name': 'NewTestFilter2',
    #'onsubmit': 'IS.Tag.newTagSubmit()',
    'csrfmiddlewaretoken': token
    # 'ui-button ui-widget ui-state-default ui-corner-all ui-button-text-only': 'submit'
}

In [14]:
resp = session.get(
    url, 
    data=payload, 
    headers=dict(referer=url)
)
resp

<Response [200]>

In [15]:
soup = BeautifulSoup(resp.text, 'lxml')
soup.find('title')

<title> Tag Management | InsightSquared</title>

In [16]:
soup.findAll('input')

[<input autocomplete="off" class="search-input" data-bind="hasFocus: search_has_focus" id="search" placeholder="Search" type="text"/>,
 <input data-bind="textInput: dashboard_name" maxlength="255"/>,
 <input autofocus="autofocus" data-bind="recipientAutocomplete: recipients, recipientFilter: filter_recipients, recipientSelect: select_shareitem" id="shared_dashboard_person" name="person" placeholder="Employee or Team" size="20" type="text"/>,
 <input data-bind="checked:set_as_default" id="shared_dashboard_set_default" name="set_default" type="checkbox"/>,
 <input id="send_report_to" name="send_report_to" type="text"/>,
 <input id="send_report_weekly_sunday" type="checkbox" value="7"/>,
 <input id="send_report_weekly_monday" type="checkbox" value="1"/>,
 <input id="send_report_weekly_tuesday" type="checkbox" value="2"/>,
 <input id="send_report_weekly_wednesday" type="checkbox" value="3"/>,
 <input id="send_report_weekly_thursday" type="checkbox" value="4"/>,
 <input id="send_report_week

In [31]:
# soup.findAll('form', attrs={'method': 'post'})
soup.findAll('form')

[<form onsubmit="return false;">\n<table>\n<tr>\n<td>\n<label>Rename</label>\n</td>\n<td>\n<input data-bind="textInput: dashboard_name" maxlength="255"/>\n</td>\n</tr>\n</table>\n</form>,
 <form onsubmit="return false;">\n<table>\n<tr>\n<td>\n<label>Dashboard</label>\n</td>\n<td>\n<span data-bind="text:dashboard_name" id="shared_dashboard_name"></span>\n</td>\n</tr>\n<!-- ko if: shareitem_shared_with_list().length -->\n<tr>\n<td colspan="2">\n<div class="shared_dashboard_with">\n<p>Currently shared with...</p>\n<ul class="shared_dashboard_persons selection tags person_names" data-bind="foreach:shareitem_shared_with_list">\n<li class="tag " data-bind="click:$parent.unselect_shareitem">\n<div class="label">\n<a data-bind="text:name"></a>\n</div>\n</li>\n</ul>\n</div>\n<hr/>\n</td>\n</tr>\n<!-- /ko -->\n<tr>\n<td>\n<label for="person">Share with</label>\n</td>\n<td>\n<input autofocus="autofocus" data-bind="recipientAutocomplete: recipients, recipientFilter: filter_recipients, recipientSel

In [19]:
session2 = requests.session()

In [20]:
login_url = 'https://clients.insightsquared.com/login/'
new = session2.get(login_url)

In [21]:
new_soup = BeautifulSoup(new.text, 'lxml')

In [22]:
new_soup.findAll('form', attrs={'method': 'post'})

[<form action="#" class="right_label login shadow" enctype="multipart/form-data" id="login_form" method="post">\n<input name="csrfmiddlewaretoken" type="hidden" value="saevPswZkF5gHXame6qY1D6JG3quRSlK"/>\n<div class="login_logo"></div>\n<div class="form_content">\n<ul>\n<li>\n<div class="container"> <!-- simple_container.html -->\n<input class="field text" id="id__email" name="id__email" onblur="this.placeholder='Email'" onfocus="this.placeholder=''" placeholder="Email" type="text" value=""/>\n<label class="input_label" for="id__email">Email</label>\n<!-- errors.html -->\n<!-- /errors.html -->\n<p class="clearfix"></p>\n</div> <!-- /simple_container.html -->\n</li>\n<li>\n<div class="container"> <!-- simple_container.html -->\n<input autocomplete="off" class="field password" id="id__password" name="id__password" onblur="this.placeholder='Password'" onfocus="this.placeholder=''" placeholder="Password" type="password" value=""/>\n<label class="input_label" for="id__password">Password</la

# doing process as lxml

In [ ]:
#lxml html
#login_tree = html.fromstring(login_resp.text)
#authenticity_token_tree = login_tree.xpath("//input[@name='csrfmiddlewaretoken']/@value")[0]

In [ ]:
tree = html.fromstring(result.content)
bucket_elems = tree.findall(".//span[@class='repo-name']/")
bucket_names = [bucket.text_content.replace("\n", "").strip() for bucket in bucket_elems]

print bucket_names